## 1. Импорт  библиотек

In [1]:
import random
import math
import copy

import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import os
import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

import torch
from kan import KAN

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
#%load_ext autotime

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_23104\1229217323.py:27: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#from tools import KAN_es
from algos import get_KAN_dataset_3_output, vector_pred_KAN_3_output, multi_exp, create_XY_data, train_KAN, train_NN_3_output, alg_keras_mlp_3_output, app_stat_NN_3_output

## 3. Обучение нейронных сетей

### 3.0. Загрузка наборов данных

In [3]:
'''
initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_vld.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_vld.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_vld.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
'''
# !Some troubles with *.csv naming!

initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_pro.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_pro.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_pro.csv")

In [4]:
data_col_names = initial_trn_data_udus.columns

### 3.1. Шкалирование данных

In [5]:
mm_scaler = MinMaxScaler()
mm_scaler.fit(pd.concat([initial_trn_data_fdfs,
                         initial_trn_data_udfs, 
                         initial_trn_data_udus],
                        axis=0, sort=False, ignore_index=True))

MinMaxScaler()

In [6]:
'''
scaled_trn_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names) 
scaled_vld_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names)
scaled_tst_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names)

scaled_trn_data_udfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_udfs),columns=data_col_names)
scaled_vld_data_udfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_udfs),columns=data_col_names)
scaled_tst_data_udfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_udfs),columns=data_col_names)

scaled_trn_data_udus = pd.DataFrame(mm_scaler.transform(initial_trn_data_udus),columns=data_col_names)
scaled_vld_data_udus = pd.DataFrame(mm_scaler.transform(initial_vld_data_udus),columns=data_col_names)
scaled_tst_data_udus = pd.DataFrame(mm_scaler.transform(initial_tst_data_udus),columns=data_col_names)
'''
trn = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names).astype('float32')
vld = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names).astype('float32')
tst = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names).astype('float32')

### Мульти-эксперимент с КАН

In [10]:
l_algos_names=['GMT_KAN_3o_hn1', 'GMT_KAN_3_0_hn2', 'GMT_KAN_3o_hn3']

l_algos=[vector_pred_KAN_3_output, vector_pred_KAN_3_output, vector_pred_KAN_3_output]

Data = (trn, vld, tst)

mult_data = [Data for i in range(3)]

l_geophysical_method = ['GMT' for i in range(3)]

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(3)]

l_kwargs=[{'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':2, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':3, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          ]

l_metrics_names=['rmse', 'mae', 'mape', 'r2'] #['mae', 'rmse', 'r2', 'mape']

num_iter=25

In [11]:
full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

--- Processing GMT_KAN_3o_hn1
iter: 1


trn_ls: 2.47e-01 | vl_ls: 2.44e-01 | e_stop: 25/25 | tst_ls: 2.47e-01 | reg: 3.45e+01 :   7%|▎   | 36/500 [04:08<53:28,  6.92s/it]


Early stopping criteria raised


c:\Users\Gavriil\VS_projects\KAN_geo\.venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


_vector_Y_tst:(1000, 3)
_vector_Y_pred:(1000, 3)
iter: 2


trn_ls: 2.64e-01 | vl_ls: 2.66e-01 | e_stop: 3/25 | tst_ls: 2.63e-01 | reg: 3.01e+01 :   2%|     | 10/500 [01:04<52:37,  6.44s/it]


KeyboardInterrupt: 

In [1]:
full_df

NameError: name 'full_df' is not defined

In [ ]:
full_df.to_excel('full_metrics_4.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [ ]:
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel('aggr_metrics_4.xlsx')
aggr_df

### Мульти-эксперимент
#### 4-5 скрытых нейронов

In [ ]:
l_algos_names=['GMT_KAN_3o_hn4', 'GMT_KAN_3_0_hn5']

l_algos=[vector_pred_KAN_3_output, vector_pred_KAN_3_output]

Data = (trn, vld, tst)

mult_data = [Data for i in range(2)]

l_geophysical_method = ['GMT' for i in range(2)]

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(2)]

l_kwargs=[{'K':3, 'hidden_neurons':4, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':5, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0}
          ]

l_metrics_names=['rmse', 'mae', 'mape', 'r2'] #['mae', 'rmse', 'r2', 'mape']

num_iter=25

In [ ]:
full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [ ]:
full_df

NameError: name 'full_df' is not defined

In [ ]:
full_df.to_excel('full_metrics_4_1.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [ ]:
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel('aggr_metrics_4_1.xlsx')
aggr_df

### Мульти-эксперимент с MLP

In [15]:
model, history_df, n_epochs = train_NN_3_output(trn, vld, 'GMT', samples_number='all',randomseed=2, 
             model_name_template='?')

Start train model: ?_GMT_all_['H1_8', 'H2_8', 'H3_8']_rs2
Number of epochs for training NN model:6337


In [16]:
history_df

,loss,val_loss
0,0.121539,0.097786
1,0.096394,0.093653
2,0.092537,0.090229
3,0.089301,0.087326
4,0.086537,0.084832
...,...,...
6332,0.019257,0.018730
6333,0.020795,0.019512
6334,0.020202,0.018295
6335,0.019958,0.020861


In [27]:
def train_NN_3_output(trn_data, vld_data, 
             geophysical_method, samples_number, randomseed, 
             model_name_template,
             l_output_parameter=['H1_8', 'H2_8', 'H3_8'],
             hidden_neurons = 32,
             learning_rate=0.1,
             momentum=0.5,
             tol=0.001,
             n_iter_no_change=500,
             max_epochs=50000,
             rel_batch_size=0.05
             ):
    
    _trn_data = copy.deepcopy(trn_data) 
    _vld_data = copy.deepcopy(vld_data)
    _geophysical_method = copy.deepcopy(geophysical_method)
    _l_output_parameter = copy.deepcopy(l_output_parameter)    
    _samples_number = copy.deepcopy(samples_number)  
    _randomseed = copy.deepcopy(randomseed)
    _model_name_template = copy.deepcopy(model_name_template)
    
    
    _model_name = (_model_name_template + "_" + 
                   _geophysical_method + "_" + 
                   str(_samples_number) + "_" +
                   str(_l_output_parameter) + "_rs" +
                   str(_randomseed))
    
    ### Create input-output data
    
    _dataset_sizes = {"all":"all", 3500:1000, 1750:500, 1000:300, 700:200, 350:100, 175:50}
    _X_trn, _Y_trn = create_XY_data(_trn_data, _l_output_parameter[0], _geophysical_method, _samples_number)
    _X_vld, _Y_vld = create_XY_data(_vld_data, _l_output_parameter[0], _geophysical_method, _dataset_sizes[_samples_number])

    for _output_parameter in _l_output_parameter[1:]:
        _, _Y_trn_new = create_XY_data(_trn_data, _output_parameter, _geophysical_method, _samples_number)
        _, _Y_vld_new = create_XY_data(_vld_data, _output_parameter, _geophysical_method, _dataset_sizes[_samples_number])

        _Y_trn = pd.concat([_Y_trn, _Y_trn_new], axis=1)
        _Y_vld = pd.concat([_Y_vld, _Y_vld_new], axis=1)

    ### Create model

    tf.keras.utils.set_random_seed(_randomseed)

    _model = tf.keras.models.Sequential() 
    _model.add(tf.keras.layers.Dense(hidden_neurons, activation=tf.nn.sigmoid))
    _model.add(tf.keras.layers.Dense(3,  activation=None))

    _optimizer = keras.optimizers.SGD(learning_rate = learning_rate,#0.001,
                                      momentum = momentum,#0.5,
                                      nesterov=True)

    _model.compile(_optimizer, loss=tf.keras.losses.MeanSquaredError())
    
    _early_stopping = EarlyStopping(monitor='val_loss',
                                    min_delta=tol * rel_batch_size,#0.0001, 
                                    patience=n_iter_no_change,#500,
                                    restore_best_weights=True)

    ### Train model
    print("Start train model:", _model_name)

    _history = _model.fit(x=_X_trn, y=_Y_trn,
                          batch_size=math.ceil(len(_X_trn) * rel_batch_size),
                          epochs=max_epochs, #100000,
                          verbose=0,
                          callbacks=[_early_stopping],
                          validation_data=(_X_vld, _Y_vld))

    _n_epochs = len(_history.history['loss'])
    
    print('Number of epochs for training NN model:' + str(_n_epochs))
    
    ### Save model
    
    #_model.save(_dir_path+"/"+ _model_name+".keras")
    
    _history_df = pd.DataFrame(_history.history)
    #plot_history_NN(_history_df, _model_name, _dir_path)
    
    #_history_df.to_csv(_dir_path+"/"+ _model_name + '_history.csv')
    
    return _model, _history_df, _n_epochs

In [32]:
def app_stat_NN_3_output(model, tst_data, 
                geophysical_method, samples_number, randomseed, 
                file_name_template, file_name_suffix, l_output_parameter=['H1_8', 'H2_8', 'H3_8']):
    
    _tst_data = copy.deepcopy(tst_data) 
    _geophysical_method = copy.deepcopy(geophysical_method)
    _l_output_parameter = copy.deepcopy(l_output_parameter)
    _samples_number = copy.deepcopy(samples_number) 
    _randomseed = copy.deepcopy(randomseed)    
    _file_name_template = copy.deepcopy(file_name_template)
    _file_name_suffix = copy.deepcopy(file_name_suffix)    
    #_dir_path = copy.deepcopy(dir_path)

    _file_name =  (_file_name_template + "_" + 
                   _geophysical_method + "_" + 
                   str(_samples_number) + "_" +
                   str(_l_output_parameter) + "_rs" +
                   str(_randomseed) + "_" + 
                   _file_name_suffix)

    ### Create input-output data
    
    _X_tst, _Y_tst = create_XY_data(_tst_data, _l_output_parameter[0], _geophysical_method, "all")

    for _output_parameter in _l_output_parameter[1:]:
        _, _Y_tst_new = create_XY_data(_tst_data, _output_parameter, _geophysical_method, _samples_number)
        
        _Y_tst = pd.concat([_Y_tst, _Y_tst_new], axis=1)
    ### Apply and calculate statistics
    
    print(1)
    print(_Y_tst.shape)
    _Y_pred = model[0].predict(_X_tst)
    print(2)
    print(_Y_pred.shape, _Y_tst.shape)
    print(_Y_pred)
    _mae = round(mean_absolute_error(_Y_tst, _Y_pred), 5)
    _rmse = round(mean_squared_error(_Y_tst, _Y_pred, squared = False), 5)
    _r2 = round(r2_score(_Y_tst, _Y_pred), 5)
    _mape = round(mean_absolute_percentage_error(_Y_tst, _Y_pred), 5)



    #_statistics_DF = pd.DataFrame({'epochs': model[2]-500, 'MAE': _mae, 'RMSE':_rmse, 'R2': _r2}, 
    #                              index=[_file_name])
    
    #print(_statistics_DF)
    
    #_statistics_DF.to_csv(_dir_path+"/"+ _file_name + '.csv')
    
    #return _statistics_DF
    return [_rmse, _mae, _mape, _r2]


In [34]:
app_stat_NN_3_output([model,], 
                     tst,
                      'GMT',
                      "all", 
                      2, "stat_udus", "?")

1
(1000, 3)
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 874us/step
2
(1000, 3) (1000, 3)
[[0.78669816 0.44820255 0.6769546 ]
 [0.28832808 0.4587307  0.40797845]
 [0.33949313 0.5509436  0.49549013]
 ...
 [0.45265752 0.55921775 0.7530388 ]
 [0.01869717 0.48821986 0.40506923]
 [0.4583342  0.85183066 0.44617385]]


c:\Users\Gavriil\VS_projects\KAN_geo\.venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[0.10458, 0.08311, 75661600000000.0, 0.85159]

In [35]:
alg_keras_mlp_3_output(trn, vld, tst, 'GMT', randomseed=1)

Start train model: ?_GMT_all_['H1_8', 'H2_8', 'H3_8']_rs1


KeyboardInterrupt: 

In [8]:
l_algos_names=['GMT_MLP_16', 'GMT_MLP_32', 'GMT_MLP_64', 'GMT_MLP_128',
               'GMT_MLP_256', 'GMT_MLP_512', 'GMT_MLP_1024']

l_algos=[alg_keras_mlp_3_output, alg_keras_mlp_3_output, alg_keras_mlp_3_output, alg_keras_mlp_3_output, alg_keras_mlp_3_output, alg_keras_mlp_3_output, alg_keras_mlp_3_output]

Data = (trn, vld, tst)

mult_data = [Data for i in range(7)]

l_geophysical_method = ['GMT' for i in range(7)]

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(7)]

l_kwargs=[{hidden_neurons=16, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {hidden_neurons=32, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {hidden_neurons=64, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {hidden_neurons=128, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {hidden_neurons=256, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {hidden_neurons=512, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {hidden_neurons=1024, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05}
          ]

l_metrics_names=['rmse', 'mae', 'mape', 'r2'] #['mae', 'rmse', 'r2', 'mape']

num_iter=25

In [9]:
full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

--- Processing GMT_MLP_16
iter: 1
Start train model: ?_GMT_all_['H1_8', 'H2_8', 'H3_8']_rs1
Number of epochs for training NN model:12366
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
iter: 2
Start train model: ?_GMT_all_['H1_8', 'H2_8', 'H3_8']_rs2


c:\Users\Gavriil\VS_projects\KAN_geo\.venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
full_df

NameError: name 'full_df' is not defined

In [ ]:
full_df.to_excel('full_metrics_4_NN.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [ ]:
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel('aggr_metrics_4_NN.xlsx')
aggr_df